## Tutorial sobre superficies de clasificación y margen en SVM
Código adaptado de "Python Data Science Handbook" y la documentación oficial de scikit-learn

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.colors import ListedColormap

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams.update({'font.size': 14})
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

### Generación y visualización del conjunto de datos

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1, random_state = 0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap=cm_bright);

### Visualizador de superficies de decisión de un SVM

In [ ]:
import numpy as np

def plot_decision_function(model, X, y, ax):
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap=cm_bright);
    
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    ax.contour(X, Y, P, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
    
    ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], s=300, linewidth=1, facecolors='none', edgecolors='k')
    ax.set_xlabel('x')
    ax.set_ylabel('y');

### Entrenamiento y visualización de un soft-margin SVM

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear', C=1E0).fit(X, y)

ax = plt.gca()
plot_decision_function(model,X,y,ax)

### Proyección y visualización en 3D

In [ ]:
from mpl_toolkits import mplot3d

z = np.exp(-(X ** 2).sum(1))
ax = plt.subplot(projection='3d')
ax.scatter3D(X[:, 0], X[:, 1], z, c=y, s=50, cmap=cm_bright)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');

### Entrenamiento y visualización de un kernel-SVM

In [ ]:
model = SVC(kernel='rbf', C=1E6, gamma='auto')
model.fit(X, y)

ax = plt.gca()
plot_decision_function(model,X,y,ax)

### Más ejemplos de kernel-SVM

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=100, centers=2, random_state=0, cluster_std=0.6)

In [ ]:
model = SVC(kernel='rbf', C=1E1)
model.fit(X, y);

In [ ]:
ax = plt.gca()
plot_decision_function(model,X,y,ax);

### Visualización de superficies de múltiples kernels
Código adaptado de https://gist.github.com/WittmannF/60680723ed8dd0cb993051a7448f7805

In [ ]:
from sklearn.datasets import make_moons, make_circles

names = ["Linear SVM", "RBF SVM", "Poly SVM"]
classifiers = [SVC(kernel="linear", C=0.025),SVC(gamma=2, C=1),SVC(kernel="poly", C=0.025)]
datasets = [make_blobs(n_samples=100, centers=2, random_state=0, cluster_std=0.6), make_moons(noise=0.3, random_state=0), make_circles(noise=0.2, factor=0.5, random_state=1)]

In [ ]:
figure = plt.figure(figsize=(30, 18))
current_subplot = 1

for ds in datasets:
    
    X, y = ds
  
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, .02), np.arange(y_min, y_max, .02))

    ax = plt.subplot(len(datasets), len(classifiers) + 1, current_subplot)
  
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    current_subplot += 1

    for name, clf in zip(names, classifiers):
        
        clf.fit(X, y)
        
        ax = plt.subplot(len(datasets), len(classifiers) + 1, current_subplot)
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=plt.cm.RdBu, alpha=.6)
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
      
        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_title(name)
        current_subplot += 1

figure.subplots_adjust(left=.02, right=.98)
plt.show()

### Algunas conclusiones
* SVM básicos entrenados en conjuntos de datos no lineales carecen de sentido práctico.
* Vectores de soporte en _kernel_-SVM entregan información relevante sobre la forma de la superficie de clasificación.
* Es posible comprobar empíricamente que distintos _kernels_ generan superficies de clasificación muy distintas, por lo que es encesario elegir el más adecuado a la tarea y los datos.